# init

In [1]:
import pygame
import numpy as np
import numpy.linalg as lin
import sys
import importlib
import itertools
import time

import Clipping
from Geometry import *
from camera import Camera
from draw import *
from inputs import input_update

# reload modules

In [90]:

importlib.reload(Clipping)
importlib.reload(sys.modules['Geometry'])
importlib.reload(sys.modules['vec'])
importlib.reload(sys.modules['util'])
from Geometry import *

importlib.reload(sys.modules['camera'])
from camera import Camera
importlib.reload(sys.modules['inputs'])
from inputs import input_update

importlib.reload(sys.modules['draw'])
from draw import *



# build shapes

In [2]:
from colors import *
    
d = 3 #set dimensionality
cube_face_colors = [RED,YELLOW,GREEN,CYAN,MAGENTA, WHITE]
tess_face_colors = cube_face_colors + [ORANGE,PURPLE]

cube_shape = build_cube(d)


if d == 4:
    for face, color in zip(cube_shape.faces,tess_face_colors):
        face.color = color
    shapes = []
    for i in range(4):
        cube = cube_shape.copy()
        cube.update(pos = 2*np.eye(d)[i])
        shapes.append(cube)
        cube = cube_shape.copy()
        cube.update(pos = -2*np.eye(d)[i])
        shapes.append(cube)
    
    #shapes = [cube_shape] #single cube
    #cube_shape.transparent = True
if d == 3:
    for face, color in zip(cube_shape.faces,cube_face_colors):
        face.color = color
    cube1 = cube_shape.copy()
    cube1.update(pos=np.array([2,0,0]))

    cube2 = cube_shape.copy()
    cube2.update(pos=np.array([-2,0,0]))

    cube3 = cube_shape.copy()
    cube3.update(pos=np.array([0,0,2]))

    cube4 = cube_shape.copy()
    cube4.update(pos=np.array([0,0,-2]))

    cube5 = cube_shape.copy()
    cube5.update(pos=np.array([0,-2,0]))
    
    shapes = [cube1,cube2,cube3,cube4,cube5]
    shapes[0].transparent = True
for i,shape in enumerate(shapes):
    shape.name = 'shape ' + str(i)

shape_combinations = list(itertools.combinations(shapes,2))

# main loop

Would like draw.py to have clearer distinctions between the various spaces. The layers are
1. The d-dim local spaces that shape.verts_ref, ref_frame, and face.normal_ref etc live in
2. The d-dim world space that rotated, scaled and positioned shapes (and faces) and cameras live
3. The d-dim space rotated and translated wrt to a camera
4. The (d-1)-dim space that shapes are projected on to
5. The (d-1)-dim graphics backend (openGL)

It would be nice if we could organize code according to these layers.

Does clipping happen in 2 or 3? Where should it happen?

In [4]:

t = 0
#size = width, height = 1100, 700
size = width, height = 1800, 900

pygame.init()

if d == 3:
    draw_class = DrawOpenGL2D(pygame,size,draw_origin=np.zeros([d-1]))
    camera = Camera(pos = np.array([0.8,2,0.8]),draw_class = draw_class)
    camera.update_rot_matrix(0,-1,-3*np.pi/4)
    camera.update_rot_matrix(1,-1,-np.pi/4)
if d == 4:
    draw_class = DrawOpenGL3D(pygame,size,
                              draw_origin=np.array([0.0,0.0,-15.0]),
                              focal=4,stereo=False)
    if draw_class.stereo:
        draw_class.view_radius = 4
    else:
        draw_class.view_radius = 6

    #draw_class.stereo_view_angles = [[-30,0],[0,0]]
    camera = Camera(pos = np.array([0,0,0,0]),draw_class = draw_class)
    #camera.look_at(np.array([2,2,2,2]))
    #camera = Camera(pos = np.array([0.8,2,0.8,0.8]),draw_class = draw_class)
    #camera.update_rot_matrix(0,-1,-3*np.pi/4)
    #camera.update_rot_matrix(2,-1,-np.pi/4)
    #camera.update_rot_matrix(1,-1,-np.pi/4)
#camera.update_rot_matrix(2,3,-np.pi/4)
camera.clipping = True
quit = False
redraw = True
timer = time.time()
frame = 0
while not quit:
    
    if redraw:
        draw_class.draw(camera,shapes)
#         if d == 4:
#             draw_class.draw_lines_3d(cube_lines_test,GREEN,draw_origin = draw_class.draw_origin + )
#             draw_class.draw_lines_3d(cube_lines_test,GREEN)
        pygame.display.flip()
        redraw = False

    #this is here so that we draw on the first frame
    redraw, quit = input_update(camera, draw_class, shapes) 
    redraw = True #redraw every loop. useful for assessing framerate
    
    
    delta_t = (time.time() - timer)
    if delta_t > 2:
    #if False:
        #print('redraw:',redraw)
        try:
            #fps = 30/(time.time() - timer)
            fps = frame/delta_t
        except:
            fps = 0
        #if fps < 60:
        print(fps,'fps')
        timer = time.time()
        frame = 0
    frame = frame+1
pygame.quit()
print('game terminated')

init GL
57.210354358007365 fps
59.932976811472365 fps
59.94588253873994 fps
transparency True
59.426282180265446 fps
59.95003810925224 fps
59.45221063419505 fps
59.941870318050704 fps
59.96009383155565 fps
59.9313853948912 fps
59.9600224011791 fps
59.95675821456931 fps
59.92767479662406 fps
59.929030538872105 fps
59.97406173992247 fps
59.44236889050168 fps
59.93644539335086 fps
game terminated


In [ ]:
pygame.quit()